In [ ]:
!pip install pymorphy2 razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=220b402667b7b57f27d9aad82b564bcddd8b2376344c33b12c7cf2d382ba7c7b
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import pandas as pd
import re
import pymorphy2
import nltk

from razdel import tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/"

df_ppc = pd.read_csv(data_path + "purchases_participants_contracts.csv", delimiter = ";", usecols = ["id", "lot_name", "supplier_inn"])
df_kpgz = pd.read_csv(data_path + "kpgz.csv", delimiter = ";", usecols = [0, 1])

df_ppc.dropna(inplace = True)
df_kpgz.columns = ["Код КПГЗ", "Наименование КПГЗ"]
df_kpgz.drop(index = [0, 1, 2], inplace = True)

In [ ]:
morph = pymorphy2.MorphAnalyzer()

nltk.download('stopwords')
stopwords_russian = stopwords.words('russian')

def substitute_tokenize_lemmatize(text):
    text = text.lower()
    text = re.sub(r"[^\w0-9]", " ", text)
    tokens = list(tokenize(text))
    return [morph.parse(t.text)[0].normal_form for t in tokens if t.text not in stopwords_russian]

def check_intersection(list1, list2):
    for value in list1:
        if value in list2:
            return True
        else:
            return False
        
def count_intersections(list1, list2):
    count = 0
    for value in list1:
        if value in list2:
            count += 1
    return count

def codes_getter(tokens):
    last_count = 0
    for i in range(len(df_kpgz1)):
        if check_intersection(tokens, df_kpgz1.iloc[i, 2]) == True and count_intersections(tokens, df_kpgz1.iloc[i, 2]) > last_count:
            last_count = count_intersections(tokens, df_kpgz1.iloc[i, 2])
            return(df_kpgz1.iloc[i, 0])#, last_count, count_intersections(tokens, df_kpgz.iloc[i, 2]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df_ppc["Tokens"] = df_ppc["lot_name"].progress_apply(lambda x: substitute_tokenize_lemmatize(x))

100%|██████████| 966313/966313 [30:16<00:00, 532.03it/s]


In [ ]:
df_kpgz["Tokens"] = df_kpgz["Наименование КПГЗ"].progress_apply(lambda x: substitute_tokenize_lemmatize(x))

100%|██████████| 17531/17531 [00:23<00:00, 731.52it/s] 


In [ ]:
df_ppc["Code"] = df_ppc["Tokens"].progress_apply(lambda x: codes_getter(x))

  1%|          | 10696/966313 [07:59<11:54:02, 22.31it/s]


KeyboardInterrupt: ignored

In [ ]:
df_ppc.head()

In [ ]:
df_ppc.to_csv('ppc_codes.csv')

In [ ]:
purchases = pd.read_csv(data_path + "purchases.csv", delimiter = ";")

In [ ]:
lot_name_unique = pd.DataFrame()
lot_name_unique['lot'] = purchases['lot_name'].unique()
lot_name_unique.dropna(inplace = True)
lot_name_unique

,lot
0,"Услуги по дезинфекции, дезинсекции и дератизации"
1,Услуги по техническому обслуживанию приборов у...
2,Принадлежности для музыкальных инструментов пр...
3,Текущий ремонт административных зданий....
4,ОКПД 2
...,...
157906,Масло сливочное;Сыры твердые;Продукция молочна...
157907,"Шпатлевки;Изделия пластмассовые строительные, ..."
157908,Устройства запоминающие;Аппаратура коммуникаци...
157909,Пакеты для упаковки готовых изделий;Тряпки для...


In [ ]:
lot_name_unique["Tokens"] = lot_name_unique["lot"].progress_apply(lambda x: substitute_tokenize_lemmatize(x))

100%|██████████| 157910/157910 [17:53<00:00, 147.04it/s]


In [ ]:
lot_name_unique

,lot,Tokens
0,"Услуги по дезинфекции, дезинсекции и дератизации","[услуга, дезинфекция, дезинсекция, дератизация]"
1,Услуги по техническому обслуживанию приборов у...,"[услуга, технический, обслуживание, прибор, уч..."
2,Принадлежности для музыкальных инструментов пр...,"[принадлежность, музыкальный, инструмент, прочий]"
3,Текущий ремонт административных зданий....,"[текущий, ремонт, административный, здание]"
4,ОКПД 2,"[окпд, 2]"
...,...,...
157906,Масло сливочное;Сыры твердые;Продукция молочна...,"[масло, сливочный, сыр, твёрдый, продукция, мо..."
157907,"Шпатлевки;Изделия пластмассовые строительные, ...","[шпатлёвка, изделие, пластмассовый, строительн..."
157908,Устройства запоминающие;Аппаратура коммуникаци...,"[устройство, запоминать, аппаратура, коммуника..."
157909,Пакеты для упаковки готовых изделий;Тряпки для...,"[пакет, упаковка, готовый, изделие, тряпка, мы..."


In [ ]:
def codes_getter(tokens):
    last_count = 0
    for i in range(len(df_kpgz1)):
        if check_intersection(tokens, df_kpgz1.iloc[i, 2]) == True and count_intersections(tokens, df_kpgz1.iloc[i, 2]) > last_count:
            last_count = count_intersections(tokens, df_kpgz1.iloc[i, 2])
            return(df_kpgz1.iloc[i, 0])#, last_count, count_intersections(tokens, df_kpgz.iloc[i, 2]))

In [ ]:
df_kpgz1 = pd.read_csv(data_path + "kpgz1.csv", delimiter = ",", usecols =[1,2])
df_kpgz1.columns = ["Код КПГЗ", "Наименование КПГЗ"]
df_kpgz1

,Код КПГЗ,Наименование КПГЗ
0,01.03.02,СИСТЕМЫ КОНТРОЛЯ ДОСТУПА И ОБОРУДОВАНИЕ ДЛЯ НИХ
1,01.06.02,ОБОРУДОВАНИЕ УЧЕБНОЕ ЭЛЕКТРОННОЕ
2,01.07.04,ОБОРУДОВАНИЕ СЦЕНИЧЕСКОЕ
3,01.01.13,ДЕТСКОЕ ПИТАНИЕ ДЛЯ РАЗДАЧИ ЧЕРЕЗ МОЛОЧНО-РАЗД...
4,01.11.10,КОНСТРУКЦИИ БЫСТРОВОЗВОДИМЫЕ
...,...,...
692,01.25.36,КОМПЛЕКСЫ РОБОТОТЕХНИЧЕСКИЕ ПРОМЫШЛЕННЫЕ
693,01.23.31,ЛЕЖАКИ ПАРКОВЫЕ
694,02.12.18,"МОНТАЖ, УСТАНОВКА, СБОРКА АВТОМАТИЗИРОВАННОЙ С..."
695,02.27.06,РАЗВИТИЕ И МОДЕРНИЗАЦИЯ ИНФОРМАЦИОННЫХ СИСТЕМ ...


In [ ]:
df_kpgz1["Tokens"] = df_kpgz1["Наименование КПГЗ"].progress_apply(lambda x: substitute_tokenize_lemmatize(x))


100%|██████████| 697/697 [00:00<00:00, 1235.67it/s]


In [ ]:
df_kpgz1

,Код КПГЗ,Наименование КПГЗ,Tokens
0,01.03.02,СИСТЕМЫ КОНТРОЛЯ ДОСТУПА И ОБОРУДОВАНИЕ ДЛЯ НИХ,"[система, контроль, доступ, оборудование]"
1,01.06.02,ОБОРУДОВАНИЕ УЧЕБНОЕ ЭЛЕКТРОННОЕ,"[оборудование, учебный, электронный]"
2,01.07.04,ОБОРУДОВАНИЕ СЦЕНИЧЕСКОЕ,"[оборудование, сценический]"
3,01.01.13,ДЕТСКОЕ ПИТАНИЕ ДЛЯ РАЗДАЧИ ЧЕРЕЗ МОЛОЧНО-РАЗД...,"[детский, питание, раздача, молочный, раздаточ..."
4,01.11.10,КОНСТРУКЦИИ БЫСТРОВОЗВОДИМЫЕ,"[конструкция, быстровозводимый]"
...,...,...,...
692,01.25.36,КОМПЛЕКСЫ РОБОТОТЕХНИЧЕСКИЕ ПРОМЫШЛЕННЫЕ,"[комплекс, робототехнический, промышленный]"
693,01.23.31,ЛЕЖАКИ ПАРКОВЫЕ,"[лежак, парковый]"
694,02.12.18,"МОНТАЖ, УСТАНОВКА, СБОРКА АВТОМАТИЗИРОВАННОЙ С...","[монтаж, установка, сборка, автоматизированный..."
695,02.27.06,РАЗВИТИЕ И МОДЕРНИЗАЦИЯ ИНФОРМАЦИОННЫХ СИСТЕМ ...,"[развитие, модернизация, информационный, систе..."


In [ ]:
lot_name_unique["Code"] = lot_name_unique["Tokens"].progress_apply(lambda x: codes_getter(x))

100%|██████████| 157910/157910 [28:58<00:00, 90.83it/s] 


In [ ]:
lot_name_unique

,lot,Tokens,Code
0,"Услуги по дезинфекции, дезинсекции и дератизации","[услуга, дезинфекция, дезинсекция, дератизация]",03.02.07
1,Услуги по техническому обслуживанию приборов у...,"[услуга, технический, обслуживание, прибор, уч...",03.02.07
2,Принадлежности для музыкальных инструментов пр...,"[принадлежность, музыкальный, инструмент, прочий]",01.15.07
3,Текущий ремонт административных зданий....,"[текущий, ремонт, административный, здание]",03.07.06
4,ОКПД 2,"[окпд, 2]",None
...,...,...,...
157906,Масло сливочное;Сыры твердые;Продукция молочна...,"[масло, сливочный, сыр, твёрдый, продукция, мо...",None
157907,"Шпатлевки;Изделия пластмассовые строительные, ...","[шпатлёвка, изделие, пластмассовый, строительн...",None
157908,Устройства запоминающие;Аппаратура коммуникаци...,"[устройство, запоминать, аппаратура, коммуника...",03.07.05
157909,Пакеты для упаковки готовых изделий;Тряпки для...,"[пакет, упаковка, готовый, изделие, тряпка, мы...",None


In [ ]:
lot_name_unique.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 157910 entries, 0 to 157910
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lot     157910 non-null  object
 1   Tokens  157910 non-null  object
 2   Code    77292 non-null   object
dtypes: object(3)
memory usage: 4.8+ MB


In [ ]:
lot_name_unique.to_csv('lot_name_unique.csv')

In [ ]:
df_kpgz = pd.read_csv(data_path + "kpgz_cut3.csv", delimiter = ",", usecols =[1,2])
df_kpgz.columns = ["Код КПГЗ", "Наименование КПГЗ"]
df_kpgz

,Код КПГЗ,Наименование КПГЗ
0,01.02.09.01.09,"ПРЕПАРАТЫ, СПОСОБСТВУЮЩИЕ ПИЩЕВАРЕНИЮ (ВКЛЮЧАЯ..."
1,01.02.09.01.13,"ОБЩЕТОНИЗИРУЮЩИЕ ПРЕПАРАТЫ,A13"
2,01.02.09.03.07,"БЕТА-АДРЕНОБЛОКАТОРЫ, C07"
3,01.02.09.04.04,ПРЕПАРАТЫ ДЛЯ ЛЕЧЕНИЯ ЗУДА (ВКЛЮЧАЯ АНТИГИСТАМ...
4,01.02.09.07.07,"ВАКЦИНЫ,J07"
...,...,...
4169,01.02.10.11.18,ЛУЧЕПРОВОД ОФТАЛЬМОЛОГИЧЕСКОЙ ЛАЗЕРНОЙ СИСТЕМЫ
4170,01.02.10.48.29,РАСХОДНЫЕ МАТЕРИАЛЫ ДЛЯ ОБОРУДОВАНИЯ RESMED
4171,01.02.10.39.10,РАСХОДНЫЕ МАТЕРИАЛЫ ДЛЯ КТ-ИНЖЕКТОРОВ NEMOTO
4172,01.02.11.03.99,МЕБЕЛЬ И ОБОРУДОВАНИЕ ДЛЯ КОМПЛЕКСНОГО ОСНАЩЕН...


In [ ]:
df_kpgz["Tokens"] = df_kpgz["Наименование КПГЗ"].progress_apply(lambda x: substitute_tokenize_lemmatize(x))

100%|██████████| 4174/4174 [00:05<00:00, 746.53it/s]


In [ ]:
def codes_getter_kpgz(tokens):
    last_count = 0
    for i in range(len(df_kpgz)):
        if check_intersection(tokens, df_kpgz.iloc[i, 2]) == True and count_intersections(tokens, df_kpgz.iloc[i, 2]) > last_count:
            last_count = count_intersections(tokens, df_kpgz.iloc[i, 2])
            return(df_kpgz.iloc[i, 0])#, last_count, count_intersections(tokens, df_kpgz.iloc[i, 2]))

In [ ]:
lot_name_unique_kpgz = lot_name_unique.copy()

In [ ]:
lot_name_unique_kpgz["Code"] = lot_name_unique_kpgz["Tokens"].progress_apply(lambda x: codes_getter_kpgz(x))

100%|██████████| 157910/157910 [1:16:38<00:00, 34.34it/s]


In [ ]:
lot_name_unique_kpgz

,lot,Tokens,Code
0,"Услуги по дезинфекции, дезинсекции и дератизации","[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
1,Услуги по техническому обслуживанию приборов у...,"[услуга, технический, обслуживание, прибор, уч...",03.06.01.01.01
2,Принадлежности для музыкальных инструментов пр...,"[принадлежность, музыкальный, инструмент, прочий]",01.07.06.01.09
3,Текущий ремонт административных зданий....,"[текущий, ремонт, административный, здание]",02.06.02.02.09
4,ОКПД 2,"[окпд, 2]",None
...,...,...,...
157906,Масло сливочное;Сыры твердые;Продукция молочна...,"[масло, сливочный, сыр, твёрдый, продукция, мо...",01.01.01.06.07
157907,"Шпатлевки;Изделия пластмассовые строительные, ...","[шпатлёвка, изделие, пластмассовый, строительн...",None
157908,Устройства запоминающие;Аппаратура коммуникаци...,"[устройство, запоминать, аппаратура, коммуника...",01.06.02.01.04
157909,Пакеты для упаковки готовых изделий;Тряпки для...,"[пакет, упаковка, готовый, изделие, тряпка, мы...",01.15.08.02.01


In [ ]:
lot_name_unique_kpgz2 = lot_name_unique_kpgz.copy()
lot_name_unique_kpgz2.rename(columns={"lot": "lot_name"}, inplace= True)
lot_name_unique_kpgz2

,lot_name,Tokens,Code
0,"Услуги по дезинфекции, дезинсекции и дератизации","[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
1,Услуги по техническому обслуживанию приборов у...,"[услуга, технический, обслуживание, прибор, уч...",03.06.01.01.01
2,Принадлежности для музыкальных инструментов пр...,"[принадлежность, музыкальный, инструмент, прочий]",01.07.06.01.09
3,Текущий ремонт административных зданий....,"[текущий, ремонт, административный, здание]",02.06.02.02.09
4,ОКПД 2,"[окпд, 2]",None
...,...,...,...
157906,Масло сливочное;Сыры твердые;Продукция молочна...,"[масло, сливочный, сыр, твёрдый, продукция, мо...",01.01.01.06.07
157907,"Шпатлевки;Изделия пластмассовые строительные, ...","[шпатлёвка, изделие, пластмассовый, строительн...",None
157908,Устройства запоминающие;Аппаратура коммуникаци...,"[устройство, запоминать, аппаратура, коммуника...",01.06.02.01.04
157909,Пакеты для упаковки готовых изделий;Тряпки для...,"[пакет, упаковка, готовый, изделие, тряпка, мы...",01.15.08.02.01


In [ ]:
lot_name_unique_kpgz.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157910 entries, 0 to 157910
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lot     157910 non-null  object
 1   Tokens  157910 non-null  object
 2   Code    135850 non-null  object
dtypes: object(3)
memory usage: 4.8+ MB


1. Добавить к lot_name_unique_kpgz столбец с названием категории из kpgz (помимо столбца с кодом
2. Запустить итерацию по purchases где при совпадении purchases['lot_name'] с lot_name_unique_kpgz['lot'] добавлять соответсвующий столбик 'Code' и 'Название категории'
3. Выгрузить обновленный файл purchases

In [ ]:
lot_name_unique_kpgz2

,lot_name,Tokens,Code
0,"Услуги по дезинфекции, дезинсекции и дератизации","[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
1,Услуги по техническому обслуживанию приборов у...,"[услуга, технический, обслуживание, прибор, уч...",03.06.01.01.01
2,Принадлежности для музыкальных инструментов пр...,"[принадлежность, музыкальный, инструмент, прочий]",01.07.06.01.09
3,Текущий ремонт административных зданий....,"[текущий, ремонт, административный, здание]",02.06.02.02.09
4,ОКПД 2,"[окпд, 2]",None
...,...,...,...
157906,Масло сливочное;Сыры твердые;Продукция молочна...,"[масло, сливочный, сыр, твёрдый, продукция, мо...",01.01.01.06.07
157907,"Шпатлевки;Изделия пластмассовые строительные, ...","[шпатлёвка, изделие, пластмассовый, строительн...",None
157908,Устройства запоминающие;Аппаратура коммуникаци...,"[устройство, запоминать, аппаратура, коммуника...",01.06.02.01.04
157909,Пакеты для упаковки готовых изделий;Тряпки для...,"[пакет, упаковка, готовый, изделие, тряпка, мы...",01.15.08.02.01


In [ ]:
df_kpgz

,Код КПГЗ,Наименование КПГЗ,Tokens
0,01.02.09.01.09,"ПРЕПАРАТЫ, СПОСОБСТВУЮЩИЕ ПИЩЕВАРЕНИЮ (ВКЛЮЧАЯ...","[препарат, способствовать, пищеварение, включа..."
1,01.02.09.01.13,"ОБЩЕТОНИЗИРУЮЩИЕ ПРЕПАРАТЫ,A13","[общетонизировать, препарат, a, 13]"
2,01.02.09.03.07,"БЕТА-АДРЕНОБЛОКАТОРЫ, C07","[бета, адреноблокатор, c, 07]"
3,01.02.09.04.04,ПРЕПАРАТЫ ДЛЯ ЛЕЧЕНИЯ ЗУДА (ВКЛЮЧАЯ АНТИГИСТАМ...,"[препарат, лечение, зуд, включая, антигистамин..."
4,01.02.09.07.07,"ВАКЦИНЫ,J07","[вакцина, j, 07]"
...,...,...,...
4169,01.02.10.11.18,ЛУЧЕПРОВОД ОФТАЛЬМОЛОГИЧЕСКОЙ ЛАЗЕРНОЙ СИСТЕМЫ,"[лучепровод, офтальмологический, лазерный, сис..."
4170,01.02.10.48.29,РАСХОДНЫЕ МАТЕРИАЛЫ ДЛЯ ОБОРУДОВАНИЯ RESMED,"[расходный, материал, оборудование, resmed]"
4171,01.02.10.39.10,РАСХОДНЫЕ МАТЕРИАЛЫ ДЛЯ КТ-ИНЖЕКТОРОВ NEMOTO,"[расходный, материал, кт, инжектор, nemoto]"
4172,01.02.11.03.99,МЕБЕЛЬ И ОБОРУДОВАНИЕ ДЛЯ КОМПЛЕКСНОГО ОСНАЩЕН...,"[мебель, оборудование, комплексный, оснащение,..."


In [ ]:
df_kpgz_merge = pd.read_csv(data_path + "kpgz_cut3.csv", delimiter = ",", usecols =[1,2])
df_kpgz_merge.columns = ["Код КПГЗ", "Наименование КПГЗ"]
df_kpgz_merge.rename(columns={"Код КПГЗ": "Code"}, inplace= True)
df_kpgz_merge

,Code,Наименование КПГЗ
0,01.02.09.01.09,"ПРЕПАРАТЫ, СПОСОБСТВУЮЩИЕ ПИЩЕВАРЕНИЮ (ВКЛЮЧАЯ..."
1,01.02.09.01.13,"ОБЩЕТОНИЗИРУЮЩИЕ ПРЕПАРАТЫ,A13"
2,01.02.09.03.07,"БЕТА-АДРЕНОБЛОКАТОРЫ, C07"
3,01.02.09.04.04,ПРЕПАРАТЫ ДЛЯ ЛЕЧЕНИЯ ЗУДА (ВКЛЮЧАЯ АНТИГИСТАМ...
4,01.02.09.07.07,"ВАКЦИНЫ,J07"
...,...,...
4169,01.02.10.11.18,ЛУЧЕПРОВОД ОФТАЛЬМОЛОГИЧЕСКОЙ ЛАЗЕРНОЙ СИСТЕМЫ
4170,01.02.10.48.29,РАСХОДНЫЕ МАТЕРИАЛЫ ДЛЯ ОБОРУДОВАНИЯ RESMED
4171,01.02.10.39.10,РАСХОДНЫЕ МАТЕРИАЛЫ ДЛЯ КТ-ИНЖЕКТОРОВ NEMOTO
4172,01.02.11.03.99,МЕБЕЛЬ И ОБОРУДОВАНИЕ ДЛЯ КОМПЛЕКСНОГО ОСНАЩЕН...


In [ ]:
# df_final_purchase = pd.concat([purchases, lot_name_unique_kpgz2],keys=["lot_name"])
df_final_purchase = pd.merge(purchases, lot_name_unique_kpgz2, on='lot_name', how='outer')
df_final_purchase

,id,purchase_name,lot_name,price,customer_inn,customer_name,delivery_region,publish_date,contract_category,Tokens,Code
0,purch_3880086,Дератизация и дезинсекция,"Услуги по дезинфекции, дезинсекции и дератизации",32400.00,7825010179,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,Санкт-Петербург,2021-12-13 16:20:28.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
1,purch_3880119,Оказание услуг по дератизации и дезинсекции в...,"Услуги по дезинфекции, дезинсекции и дератизации",21429.12,7802143260,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 16:30:54.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
2,purch_3880130,Оказание услуг по дератизации и дезинсекции в ...,"Услуги по дезинфекции, дезинсекции и дератизации",9857.75,7802140301,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТ...,Санкт-Петербург,2021-12-13 16:34:24.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
3,purch_3880362,Дератизация и дезинсекция в 2022 году,"Услуги по дезинфекции, дезинсекции и дератизации",41050.00,7813124534,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 17:32:31.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
4,purch_3880464,Оказание услуг по проведению дезинсекции и дер...,"Услуги по дезинфекции, дезинсекции и дератизации",35560.44,7816050104,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 17:57:28.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01
...,...,...,...,...,...,...,...,...,...,...,...
1059559,purch_4065942,На поставку продуктов питания (масло сливочно...,Масло сливочное;Сыры твердые;Продукция молочна...,9108.64,2329020903,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛ...,Краснодарский,2022-05-31 21:00:00.000,Потребность,"[масло, сливочный, сыр, твёрдый, продукция, мо...",01.01.01.06.07
1059560,purch_4065946,Поставка шпатлевки гипсовой и ведер пластиковых,"Шпатлевки;Изделия пластмассовые строительные, ...",29550.00,5021011362,Муниципальное бюджетное общеобразовательное уч...,Московская,2022-05-31 21:12:00.977,Потребность,"[шпатлёвка, изделие, пластмассовый, строительн...",None
1059561,purch_4065957,Поставка оборудования,Устройства запоминающие;Аппаратура коммуникаци...,61100.40,7825331790,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ...,Санкт-Петербург,2022-05-31 21:49:55.000,Потребность,"[устройство, запоминать, аппаратура, коммуника...",01.06.02.01.04
1059562,purch_4065976,Поставка бытовой химии,Пакеты для упаковки готовых изделий;Тряпки для...,17609.00,2364000890,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛ...,Краснодарский,2022-05-31 23:25:00.000,Потребность,"[пакет, упаковка, готовый, изделие, тряпка, мы...",01.15.08.02.01


In [ ]:
final_purchase = pd.merge(df_final_purchase, df_kpgz_merge, on='Code', how='left')
final_purchase

,id,purchase_name,lot_name,price,customer_inn,customer_name,delivery_region,publish_date,contract_category,Tokens,Code,Наименование КПГЗ
0,purch_3880086,Дератизация и дезинсекция,"Услуги по дезинфекции, дезинсекции и дератизации",32400.00,7825010179,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,Санкт-Петербург,2021-12-13 16:20:28.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
1,purch_3880119,Оказание услуг по дератизации и дезинсекции в...,"Услуги по дезинфекции, дезинсекции и дератизации",21429.12,7802143260,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 16:30:54.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
2,purch_3880130,Оказание услуг по дератизации и дезинсекции в ...,"Услуги по дезинфекции, дезинсекции и дератизации",9857.75,7802140301,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТ...,Санкт-Петербург,2021-12-13 16:34:24.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
3,purch_3880362,Дератизация и дезинсекция в 2022 году,"Услуги по дезинфекции, дезинсекции и дератизации",41050.00,7813124534,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 17:32:31.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
4,purch_3880464,Оказание услуг по проведению дезинсекции и дер...,"Услуги по дезинфекции, дезинсекции и дератизации",35560.44,7816050104,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 17:57:28.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
...,...,...,...,...,...,...,...,...,...,...,...,...
1059559,purch_4065942,На поставку продуктов питания (масло сливочно...,Масло сливочное;Сыры твердые;Продукция молочна...,9108.64,2329020903,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛ...,Краснодарский,2022-05-31 21:00:00.000,Потребность,"[масло, сливочный, сыр, твёрдый, продукция, мо...",01.01.01.06.07,СОУСЫ НА РАСТИТЕЛЬНОМ МАСЛЕ
1059560,purch_4065946,Поставка шпатлевки гипсовой и ведер пластиковых,"Шпатлевки;Изделия пластмассовые строительные, ...",29550.00,5021011362,Муниципальное бюджетное общеобразовательное уч...,Московская,2022-05-31 21:12:00.977,Потребность,"[шпатлёвка, изделие, пластмассовый, строительн...",None,NaN
1059561,purch_4065957,Поставка оборудования,Устройства запоминающие;Аппаратура коммуникаци...,61100.40,7825331790,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ...,Санкт-Петербург,2022-05-31 21:49:55.000,Потребность,"[устройство, запоминать, аппаратура, коммуника...",01.06.02.01.04,КОММУТАЦИОННЫЕ УСТРОЙСТВА УЧЕБНЫЕ
1059562,purch_4065976,Поставка бытовой химии,Пакеты для упаковки готовых изделий;Тряпки для...,17609.00,2364000890,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛ...,Краснодарский,2022-05-31 23:25:00.000,Потребность,"[пакет, упаковка, готовый, изделие, тряпка, мы...",01.15.08.02.01,ПАКЕТЫ ПОЧТОВЫЕ БУМАЖНЫЕ


In [1]:
final_purchase

NameError: ignored

In [ ]:
final_purchase.head(300)

,id,purchase_name,lot_name,price,customer_inn,customer_name,delivery_region,publish_date,contract_category,Tokens,Code,Наименование КПГЗ
0,purch_3880086,Дератизация и дезинсекция,"Услуги по дезинфекции, дезинсекции и дератизации",32400.00,7825010179,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,Санкт-Петербург,2021-12-13 16:20:28.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
1,purch_3880119,Оказание услуг по дератизации и дезинсекции в...,"Услуги по дезинфекции, дезинсекции и дератизации",21429.12,7802143260,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 16:30:54.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
2,purch_3880130,Оказание услуг по дератизации и дезинсекции в ...,"Услуги по дезинфекции, дезинсекции и дератизации",9857.75,7802140301,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТ...,Санкт-Петербург,2021-12-13 16:34:24.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
3,purch_3880362,Дератизация и дезинсекция в 2022 году,"Услуги по дезинфекции, дезинсекции и дератизации",41050.00,7813124534,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 17:32:31.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
4,purch_3880464,Оказание услуг по проведению дезинсекции и дер...,"Услуги по дезинфекции, дезинсекции и дератизации",35560.44,7816050104,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-13 17:57:28.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
...,...,...,...,...,...,...,...,...,...,...,...,...
295,purch_3963353,Oказание услуг по уборке помещений,"Услуги по дезинфекции, дезинсекции и дератизации",97578.72,5049023047,"МКУ ""СПЕЦИАЛИЗИРОВАННАЯ СЛУЖБА ПО ВОПРОСАМ ПОХ...",Московская,2021-12-24 11:26:00.423,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
296,purch_3901987,"Оказание услуг по дератизации , дезинсекции и ...","Услуги по дезинфекции, дезинсекции и дератизации",24244.60,2301057830,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛ...,Краснодарский,2021-12-24 11:50:00.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
297,purch_3983372,оказание услуг по проведению дезинсекции и дер...,"Услуги по дезинфекции, дезинсекции и дератизации",504919.30,5038004204,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,Московская,2021-12-24 11:58:33.853,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...
298,purch_3902019,Оказание услуг по дератизации и дезинсекции в...,"Услуги по дезинфекции, дезинсекции и дератизации",15668.55,7802147900,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ ...,Санкт-Петербург,2021-12-24 11:59:02.000,Потребность,"[услуга, дезинфекция, дезинсекция, дератизация]",03.06.01.01.01,УСЛУГИ ТЕХНИЧЕСКОГО ЗАКАЗЧИКА НА ПЕРИОД ВЫПОЛН...


In [ ]:
final_purchase.to_csv('final_purchase1.csv')

In [ ]:
from google.colab import files
files.download('final_purchase1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
final_purchase['Наименование КПГЗ'].nunique()

1188

In [ ]:
final_purchase['Наименование КПГЗ'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1059564 entries, 0 to 1059563
Series name: Наименование КПГЗ
Non-Null Count   Dtype 
--------------   ----- 
917350 non-null  object
dtypes: object(1)
memory usage: 16.2+ MB


In [ ]:
final_purchase['lot_name'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1059564 entries, 0 to 1059563
Series name: lot_name
Non-Null Count    Dtype 
--------------    ----- 
1026386 non-null  object
dtypes: object(1)
memory usage: 16.2+ MB
